In [10]:
import sys, os

MARIPOSA_PATH = "/home/yizhou7/mariposa/"

sys.path.append(MARIPOSA_PATH + "src")
os.chdir(MARIPOSA_PATH)

from debugger.debugger import DbgMode
from debugger.bench_viewer import BenchViewer
from debugger.debugger_options import DebugOptions
from debugger.strainer import StrainerStatus
from benchmark_consts import *

In [11]:
qs = UNSTABLE_MARIPOSA
options = DebugOptions()

mv = BenchViewer(qs, options)

[WARN] [init] 2a3b1202bd trace slow unknown, fallback to timeout 
[WARN] [init] previous debugging attempt has failed, run with --retry-failed if needed! 
[WARN] [proof] no proofs available 
[WARN] [init] e998d08b31 trace slow unknown, fallback to timeout 
[WARN] [init] 5b7d18ab3e trace did not fail, fallback to fast_fail 
[WARN] [init] previous debugging attempt has failed, run with --retry-failed if needed! 
[WARN] [proof] no proofs available 
[WARN] [init] d4e5e77cbc trace slow unknown, fallback to timeout 


[DEBUG] [edit] proof path: dbg/e998d08b31/proofs/reseed.10397248321928259864.proof 
[DEBUG] [edit] trace path: dbg/e998d08b31/traces/reseed.5819383338623026635 


e1e4a54bd0daf1d2decf775a2c3f0f4a.pickle
[WARN] [init] 3cead21983 trace slow unknown, fallback to timeout 
[WARN] [init] previous debugging attempt has failed, run with --retry-failed if needed! 
[WARN] [proof] no proofs available 


[DEBUG] [edit] proof path: dbg/d4e5e77cbc/proofs/reseed.11448191909512973404.proof 
[DEBUG] [edit] trace path: dbg/d4e5e77cbc/traces/reseed.2822452825194262778 


26da28d6b52a6822eaa8bc037122f787.pickle
[WARN] [init] previous debugging attempt has failed, run with --retry-failed if needed! 
[WARN] [proof] no proofs available 


...

[WARN] [differ] qid constructor_accessor_axiom not found in dbg/d4e5e77cbc/orig.smt2 
[WARN] [differ] qid unnamed not found in dbg/d4e5e77cbc/orig.smt2 
[WARN] [differ] qid is-Prims.equals not found in dbg/d4e5e77cbc/orig.smt2 
[WARN] [differ] qid is-Prims.h_equals not found in dbg/d4e5e77cbc/orig.smt2 
[WARN] [differ] qid is-Prims.c_and not found in dbg/d4e5e77cbc/orig.smt2 
[WARN] [differ] qid is-Prims.c_or not found in dbg/d4e5e77cbc/orig.smt2 
[WARN] [differ] qid is-Prims.dtuple2 not found in dbg/d4e5e77cbc/orig.smt2 
[WARN] [differ] qid is-Prims.list not found in dbg/d4e5e77cbc/orig.smt2 
[WARN] [differ] qid is-FStar.Pervasives.Native.option not found in dbg/d4e5e77cbc/orig.smt2 
[WARN] [differ] qid is-FStar.Pervasives.Native.tuple2 not found in dbg/d4e5e77cbc/orig.smt2 
[WARN] [differ] qid is-FStar.Pervasives.Native.tuple3 not found in dbg/d4e5e77cbc/orig.smt2 
[WARN] [differ] qid is-FStar.Pervasives.Native.tuple4 not found in dbg/d4e5e77cbc/orig.smt2 
[WARN] [differ] qid is-FSta

[DEBUG] saving cache at d4e5e77cbc.to.report 


[WARN] [differ] qid constructor_accessor_axiom not found in dbg/3cead21983/orig.smt2 
[WARN] [differ] qid <null> not found in dbg/3cead21983/orig.smt2 


[DEBUG] saving cache at 3cead21983.to.report 


[WARN] [init] 0b497bc090 trace did not fail, fallback to fast_fail 


In [12]:
mv.status.print_status()

| category                   |   count | percentage   |
|----------------------------|---------|--------------|
| StrainerStatus.FINISHED    |     539 | 98.9 %       |
| StrainerStatus.NO_PROOF    |       4 | 0.73 %       |
| StrainerStatus.NOT_CREATED |       1 | 0.18 %       |
| StrainerStatus.NOT_TESTED  |       1 | 0.18 %       |
| total                      |     545 | 100.00 %     |


In [4]:
from utils.analysis_utils import fmt_percent

bench_total = len(mv.status.tally)
bench_finished = len(mv.status[StrainerStatus.FINISHED])
# to_fixable = 0
to_cases = dict()
ff_cases = dict()

def print_break_down(p, q):
    return f"{p}/{q} ({fmt_percent(p, q, 1)})"

for q in mv.status[StrainerStatus.FINISHED]:
    dbg = mv[q]
    if dbg.mode == DbgMode.TIMEOUT:
        to_cases[q] = q in mv.fixable
    elif dbg.mode == DbgMode.FAST_FAIL:
        ff_cases[q] = q in mv.fixable
    else:
        print("Unexpected mode", dbg.mode)
        assert False

to_fixable = sum(to_cases.values())
ff_fixable = sum(ff_cases.values())

print("Test Finished:", print_break_down(bench_finished, bench_total))
print("\tTOs:", print_break_down(len(to_cases), bench_finished))
print("\t\tTO fixable:", print_break_down(to_fixable, len(to_cases)))
print("\tFFs:", print_break_down(len(ff_cases), bench_finished))
print("\t\tFF fixable:", print_break_down(ff_fixable, len(ff_cases)))

print("Test Not Finished:", print_break_down(len(mv.status[StrainerStatus.NOT_CREATED]), bench_total))
print("No Proofs:", fmt_percent(len(mv.status[StrainerStatus.NO_PROOF]), bench_total))
print("* fixable:", print_break_down(to_fixable + ff_fixable, bench_total))

Test Finished: 536/545 (98.3%)
	TOs: 445/536 (83.0%)
		TO fixable: 276/445 (62.0%)
	FFs: 91/536 (17.0%)
		FF fixable: 51/91 (56.0%)
Test Not Finished: 1/545 (0.2%)
No Proofs: 0.73%
* fixable: 327/545 (60.0%)


In [5]:
for q in mv.status[StrainerStatus.NOT_CREATED]:
    print("./src/debugger3.py -m auto -i", mv[q].name_hash, "--create-project")

./src/debugger3.py -m auto -i ab307c9008 --create-project


In [ ]:
print("unfixable:", len(mv.unfixable))
unfixable = set([mv[q].name_hash for q in mv.unfixable])
print(len(unfixable & set(DOUBLETON_TARGETS)))

Unfixable: 209
114


In [9]:
for q in mv.status[StrainerStatus.NOT_TESTED]:
    print("./src/make_spaghet.py --not-verus -i ", mv[q].strainer.test_dir)

./src/make_spaghet.py --not-verus -i  data/projs/timeout_d4e5e77cbc/base.z3
./src/make_spaghet.py --not-verus -i  data/projs/timeout_3ce5c5e4dc/base.z3
./src/make_spaghet.py --not-verus -i  data/projs/timeout_3cead21983/base.z3
./src/make_spaghet.py --not-verus -i  data/projs/timeout_e998d08b31/base.z3


In [16]:
from utils.cluster_utils import get_sync_commands
from utils.system_utils import write_misc_script

for q in mv.status.tally:
    dbg = mv[q]
    dbg.collect_garbage()

cmds = get_sync_commands("g2001", mv.get_sync_dirs())
write_misc_script("send.sh", cmds)

[WARN] [dbg] skipped GC on unfinished project StrainerStatus.NOT_CREATED data/projs/bench_unstable/base.z3/d_komodo--verified-sha-sha256.i.dfyImpl___module.__default.lemma__InputHelper.smt2 
[WARN] [dbg] skipped GC on unfinished project StrainerStatus.NO_PROOF data/projs/bench_unstable/base.z3/d_komodo--verified-remove.gen.dfyImpl___module.__default.va__lemma__kom__smc__remove__non__addr__success.smt2 
[WARN] [dbg] skipped GC on unfinished project StrainerStatus.NOT_TESTED data/projs/bench_unstable/base.z3/fs_dice--queries-ASN1.Spec.Tag-2.smt2 
[WARN] [dbg] skipped GC on unfinished project StrainerStatus.NO_PROOF data/projs/bench_unstable/base.z3/d_fvbkv--lib-Base-BitsetLemmas.i.dfy.Impl__BitsetLemmas.__default.all__in__set__implies__all1s__uint64.smt2 
[WARN] [dbg] skipped GC on unfinished project StrainerStatus.NO_PROOF data/projs/bench_unstable/base.z3/fs_vwasm--compiler-linearizer-queries-Compiler.Linearizer-15.smt2 
[WARN] [dbg] skipped GC on unfinished project StrainerStatus.NO_P